In [ ]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(r'/content/student-mat.csv')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [ ]:
df = df[~df['G3'].isin([0, 1])]
df.shape
df = df.rename(columns={'G3': 'Grade'})

In [ ]:
def format_data(df):
    df=df.drop(columns=['school','G1','G2'])
    df=pd.get_dummies(df)
    labels = df['Grade']
    most_correlated = df.corr().abs()['Grade'].sort_values(ascending=False)
    most_correlated=most_correlated[:8]
    df = df.loc[:, most_correlated.index]
    df = df.drop(columns = ['higher_no','Grade'])
    X_train, X_test, y_train, y_test = train_test_split(df, labels,test_size = 0.25,random_state=42)
    return X_train,X_test,y_train,y_test

In [ ]:
!pip3 install pyro-ppl



In [ ]:
X_train,X_test,y_train,y_test=format_data(df)
print(y_train)



619    13
323    10
201    16
410    17
426    11
       ..
71     10
106    10
272    15
437    12
102    12
Name: Grade, Length: 474, dtype: int64


In [ ]:
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm.auto import trange, tqdm
import pyro

In [116]:
class Model(PyroModule):
    def __init__(self, h1=30, h2=15):
        super().__init__()
        self.fc1 = PyroModule[nn.Linear](6, h1)
        self.fc1.weight = PyroSample(dist.Normal(0., 1.).expand([h1, 6]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(0., 1.).expand([h1]).to_event(1))
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(dist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(dist.Normal(0., 1.).expand([h2]).to_event(1))
        self.fc3 = PyroModule[nn.Linear](h2, 1)
        self.fc3.weight = PyroSample(dist.Normal(0., 1.).expand([1, h2]).to_event(2))
        self.fc3.bias = PyroSample(dist.Normal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()
    def forward(self, x, y=None):
        x = x.reshape(-1, 6)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = self.fc3(x).squeeze()
        sigma = pyro.sample("sigma", dist.Exponential(1.))
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma), obs=y)
        return mu

In [117]:
model = Model()
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 1e-3})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

pyro.clear_param_store()
bar = trange(20000)
x_train = torch.from_numpy(np.array(X_train)).float()
y_train = torch.from_numpy(np.array(y_train)).float()
for epoch in bar:
    loss = svi.step(x_train, y_train)
    bar.set_postfix(loss=f'{loss / x.shape[0]:.3f}')

In [118]:
predictive = Predictive(model, guide=guide, num_samples=2000)
test = torch.from_numpy(np.array(X_test)).float()
preds = predictive(test)
y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
print(y_pred.shape)

(159,)


In [119]:
y_test = np.array(y_test)
print(y_test.shape)

(159,)


In [120]:
error = y_pred - y_test
mae = np.mean(abs(error))
rmse = (np.mean(error**2))**1/2
print(mae)
print(rmse)

1.8444569365783308
2.471375303597321
